# Multi-Output Models

* Using the [Energy Efficiency dataset](https://archive.ics.uci.edu/ml/datasets/Energy+efficiency)
* [Google Colab](https://colab.research.google.com/drive/1a7IjqwBuf5c2QTYrolbU4Mx8g-kKy2jz#scrollTo=NGLR70vES5Pm)

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split

print(pd.__version__)
print(tf.version.VERSION)


# Utilities

Define some utility functions

In [ ]:
def format_output(data):
    y1 = data.pop("Y1")
    y1 = np.array(y1)
    y2 = data.pop("Y2")
    y2 = np.array(y2)
    return y1, y2


def norm(x, train_stats):
    return (x - train_stats["mean"]) / train_stats["std"]


def plot_diff(y_true, y_pred, title=""):
    plt.scatter(y_true, y_pred)
    plt.title(title)
    plt.xlabel("True Values")
    plt.ylabel("Predictions")
    plt.axis("equal")
    plt.axis("square")
    plt.xlim(plt.xlim())
    plt.ylim(plt.ylim())
    plt.plot([-100, 100], [-100, 100])
    plt.show()


def plot_metrics(metric_name, title, history, ylim=5):
    plt.title(title)
    plt.ylim(0, ylim)
    plt.plot(history.history[metric_name], color="blue", label=metric_name)
    plt.plot(history.history["val_" + metric_name], color="green", label="val_" + metric_name)
    plt.show()

In [ ]:
# Download dataset
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx -o ./data/ENB2012_data.xlsx

# Prepare the data

Download the dataset and format it for training

In [ ]:
# Load data into a dataframe
URI = "./data/ENB2012_data.xlsx" # umm.... I don't have this yet?
df = pd.read_excel(URI)
df = df.sample(frac=1).reset_index(drop=True)

# Split data into training and test sets (80% train, 20% test)
train, test = train_test_split(df, test_size=0.2)
train_stats = train.describe()

# Get Y1 and Y2 as the two outputs, then format them as np arrays
train_stats.pop("Y1")
train_stats.pop("Y2")
train_stats = train_stats.transpose()
train_Y = format_output(train)
test_Y = format_output(test)

# Normalize the data
norm_train_X = norm(train)
norm_test_X = norm(test)


# Build the model

Using functional syntax so we can specify a list of outputs

In [ ]:
input_layer = Input(shape=(len(train .columns),))
first_dense = Dense(128, activation="relu")(input_layer)
second_dense = Dense(128, activation="relu")(first_dense)

y1_output = Dense(1, name="y1_output")(second_dense)
third_dense = Dense(64, activation="relu")(second_dense)
y2_output = Dense(1, name="y2_output")(third_dense)

# Define the model with the input layer and a list of output layers
model = Model(inputs=input_layer, outputs=[y1_output, y2_output])
print(model.summary())

# Configure parameters

Specify the optimizer, loss, and metrics for each output

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss={"y1_output": "mse", "y2_output": "mse"},
    metrics={
        "y1_output": tf.keras.metrics.RootMeanSquaredError(),
        "y2_output": tf.keras.metrics.RootMeanSquaredError(),
    }
)

# Train the model

In [ ]:
history = model.fit(
    norm_train_X,
    train_Y,
    epochs=500,
    batch_size=10,
    validation_data=(norm_test_X, test_Y),
)

# Evaluate the model and plot metrics

In [ ]:
loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(x=norm_test_X, y=test_Y)
print(f"Loss = {loss}, Y1_loss = {Y1_loss}, Y2_loss = {Y2_loss}, Y1_rmse = {Y1_rmse}, Y2_rmse = {Y2_rmse}")

In [ ]:
# Plot the loss and mse
Y_pred = model.predict(norm_test_X)
plot_diff(test_Y[0], Y_pred[0], "Y1")
plot_diff(test_Y[1], Y_pred[1], "Y2")
plot_metrics(metric_name="y1_output_root_mean_squared_error", title="Y1 RMSE", ylim=6)
plot_metrics(metric_name="y2_output_root_mean_squared_error", title="Y2 RMSE", ylim=7)